# 実践その壱
自分で用意したデータを使って画像分類モデルをトレーニングし、結果を評価する。

## 条件
- 分類してみたい画像を最低3カテゴリー用意する。レッスン2で使用した「グリズリー」「ブラックベア」「テディベア」はNG。各カテゴリー50個以上データサンプルがあること。
- fastaiのライブラリを使用すること


## ヒント
- レッスン2の動画（ https://www.youtube.com/watch?v=MIaAUIc87r4&list=PLPth6MvJU5iFbsl1eNOLV1jJUv2JMjdgg ） を参考にAzureのSearch APIを使ってデータをダウンロードするもよし、こちらの動画（ https://www.youtube.com/watch?v=YZ0czV4N9d0&list=PLPth6MvJU5iEG4Ql3RbKdj7fWWu4JSJmH ）Googleで検索した画像をJavascriptで落とすのも良し。一つ一つ手動でダウロードして一つずつアップロードするも良し。方法は自由です。

### その他
fastaiのバージョンは2.xを使用してください。Google Colabにデフォルトで入っているバージョンは1.0のため、まず最初にfastaiをアップグレードする必要があります。

```
! pip install --upgrade fastai==2.4.1
```

上記実行後、ランタイムを再起動します。GPUを使う設定も忘れずに。

In [ ]:
!pip install --upgrade fastai==2.4.1

In [ ]:
# fastaiのバージョンをチェックします。
# 2.x ならokです。

import fastai
fastai.__version__

In [ ]:
# ここにStudent IDを記載しておいてください。
Student_ID = "Akira Endo"


In [ ]:
# import library
from fastai.vision.all import *
import requests 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 1. 画像を準備する

まず、*data*という名前のディレクトリを作成、更にその中にカテゴリーに対応したディレクトリを作成します。
その後カテゴリーに対応したディレクトリに画像をデータをそれぞれ入れていきます。

以下に例を示します。
```
./data/black/1.jpg
./data/grizzly/4.jpg
./data/teddy/2.jpg
```

例えば```./data/black/```の中にブラックベアの画像が入ることになります。
但し今回は自分で考えたカテゴリーを使うのでblack等はそれに合った名前を使ってください。

In [ ]:
# 下記を実行することで、クマのサンプルでテストできます。
# ですが、このアサインメントをパスするには"クマ"以外の自分のデータを自分のスキルでなんとかして用意する必要があります。

! wget http://shabelab.com/datasets/bears.tar.gz
! tar xvzf bears.tar.gz
# フルーツのデータセットをダウンロードする
# !curl -L "https://public.roboflow.com/ds/EcPqLjU7Lj?key=wt5JauBX6A" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

## 2. データセットを設定し、学習を行う
以下は各セルを順番に実行していくだけでokです。

*学習のヒント:*
将来類似の問題に出くわした際は、似た様な問題を暑かったnotebooksやgithubのプロジェクトを探し、それを変更して解決できないか試してみましょう。データを変えるだけで解決できることは少ないですが、そこが出発点として使える事は多いです。

In [ ]:
# datablock
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())

In [ ]:
# prepare dataloaders
dls = bears.dataloaders("./data")

In [ ]:
# train 
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
# evaluate training
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()


In [ ]:
# save 
learn.export()

## 3. テスト
学習で使わなかった画像を一つ用意しアップロードします。  
その後、保存したモデルをロードし、どんな推論結果が出るかテストしてみましょう。

ここではtest.jpgという名前で保存しています。

In [ ]:
# 画像を一つアップロードする。
# 以下はクマの例です。アサインメントでは自分のデータに合うサンプルを用意する必要があります。

! wget https://shabelab.com/mp3/1200px-2010-kodiak-bear-1.jpg -O test.jpg


In [ ]:
# load model
learn_inf = load_learner('export.pkl')

In [ ]:
# upload image 
test_image = "test.jpg"

In [ ]:
from IPython.display import Image
Image(filename=test_image) 

In [ ]:
# test model
learn_inf.predict(test_image)